In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv


In [23]:
def str_to_set(x):
    return set(item['name'] for item in eval(x))
#     genre_set = set()
#     for item in eval(x):
#         genre_set.add(item['name'])
#     return genre_set


meta = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
meta = meta[['id', 'original_title', 'original_language', 'genres']] # 원하는 컬럼들만 뽑아냄
meta = meta.rename(columns={'id':'movieId', 'original_title':'title', 
                    'original_language':'language'})

# 영어로된 영화들만 남기기
meta = meta.loc[meta['language'] == 'en',:]
# movieId를 int타입으로 변화시켜주기
meta.movieId = pd.to_numeric(meta.movieId)
# genres의 데이터들 정제하기 -> 키워드만 뽑기
meta.genres = meta.genres.apply(str_to_set)

meta

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,movieId,title,language,genres
0,862,Toy Story,en,"{Family, Comedy, Animation}"
1,8844,Jumanji,en,"{Fantasy, Family, Adventure}"
2,15602,Grumpier Old Men,en,"{Romance, Comedy}"
3,31357,Waiting to Exhale,en,"{Romance, Comedy, Drama}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Action, Romance, Drama}"
45463,67758,Betrayal,en,"{Thriller, Action, Drama}"
45464,227506,Satana likuyushchiy,en,{}


In [15]:
# meta.loc은 DataFrame에서 자료를 조회할때 사용하는 명령어
# 행, 열 순으로 값을 넣어준다.
# 행, 열부분에 넣을 수 있는 것: 
# 1. 행 또는 열의 이름,
# 2. :(콜론) -> 전부 다
# 3. 리스트 ['title', 'language']
# 4. Series (불리언 값을 가짐.)

#meta = meta.loc[meta['language'] == 'en',:]
#(meta['language'] == 'en')

#meta.loc[4,'title']
#meta.loc[4, :] # 4행에 있는거 다 가져오겠다
#meta.loc[:, 'title'] # 모든 title들 다 가져오겠다.

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


In [30]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')

keywords.id = pd.to_numeric(keywords.id)
keywords = keywords.rename(columns={'id':'movieId'})
keywords.keywords = keywords.keywords.apply(str_to_set)

keywords

,movieId,keywords
0,862,"{rivalry, friendship, boy next door, friends, ..."
1,8844,"{giant insect, disappearance, recluse, based o..."
2,15602,"{duringcreditsstinger, fishing, best friend, o..."
3,31357,"{based on novel, interracial relationship, div..."
4,11862,"{pregnancy, contraception, midlife crisis, dau..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{play, artist, pinoy}"
46416,67758,{}
46417,227506,{}


In [32]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Family, Comedy, Animation}","{rivalry, friendship, boy next door, friends, ..."
1,8844,Jumanji,en,"{Fantasy, Family, Adventure}","{giant insect, disappearance, recluse, based o..."
2,15602,Grumpier Old Men,en,"{Romance, Comedy}","{duringcreditsstinger, fishing, best friend, o..."
3,31357,Waiting to Exhale,en,"{Romance, Comedy, Drama}","{based on novel, interracial relationship, div..."
4,11862,Father of the Bride Part II,en,{Comedy},"{pregnancy, contraception, midlife crisis, dau..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Action, Romance, Drama}",{}
32849,67758,Betrayal,en,"{Thriller, Action, Drama}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [42]:
# jaccard 유사도 구하기
dk = meta.loc[meta.title == 'The Dark Knight',:].iloc[0,:]
dkr = meta.loc[meta.title == 'The Dark Knight Rises',:].iloc[0,:]
pn = meta.loc[meta.title == 'Prom Night',:].iloc[0,:]
toy = meta.loc[meta.title == 'Toy Story',:].iloc[0,:]

# 두개의 set합치기
dk_set = dk.genres | dk.keywords
dkr_set = dkr.genres | dkr.keywords
pn_set = pn.genres | pn.keywords
toy_set = toy.genres | toy.keywords

def jaccard_similarity(s1, s2):
    if len(s1 | s2) == 0:
        return 0
    return len(s1 & s2) / len(s1 | s2)

jaccard_similarity(dk_set, pn_set)

0.022222222222222223

In [47]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')

ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


In [48]:
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# u1, u2는 series (벡터)
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    # 내적(u1_c,u2_c)/(u1_c크기,u2_c크기)
    denom = ((u1_c ** 2).sum() * (u2_c ** 2).sum()) ** 0.5
    if denom == 0:
        return 0
    else:
        return (u1_c * u2_c).sum()/denom

In [52]:
dk_rating = matrix['The Dark Knight']
pn_rating = matrix['Prom Night']

pearson_similarity(dk_rating, pn_rating)

0.773565934694095

In [58]:
def find_similar_movies(input_title, n, alpha):
    input_meta = meta.loc[meta['title'] == input_title].iloc[0]
    input_set = input_meta.genres | input_meta.keywords
    
    results = []
    
    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        
        this_meta = meta.loc[meta['title'] == this_title].iloc[0]
        this_set = this_meta.genres | this_meta.keywords
        
        jaccard = jaccard_similarity(input_set, this_set)
        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        
        score = alpha * pearson + (1-alpha) * jaccard
        results.append( (this_title, jaccard, pearson, score) )
    
    results.sort(key=lambda r: r[3], reverse=True)
    
    return results[:n]


In [62]:
res = find_similar_movies("Rocky III", 10, 0.5)
pd.DataFrame(res, columns=['title', 'jaccard', 'pearson', 'score'])

,title,jaccard,pearson,score
0,Rocky V,0.272727,0.297157,0.284942
1,Rocky II,0.333333,0.182715,0.258024
2,Rocky Balboa,0.357143,0.152681,0.254912
3,Rocky IV,0.346154,0.117354,0.231754
4,About Schmidt,0.040000,0.295264,0.167632
5,54,0.041667,0.293060,0.167363
6,Cinderella Man,0.296296,0.030373,0.163335
7,Michael,0.038462,0.287140,0.162801
8,I'll Sleep When I'm Dead,0.090909,0.223659,0.157284
9,Touch of Evil,0.034483,0.280075,0.157279


In [61]:
from collections import Counter

cntr = Counter()
cntr['a'] = 0.5
cntr['b'] = 1.3
cntr['c'] = 1.0

cntr.most_common(2)

,title,jaccard,pearson,score
0,Wild Wild West,0.032258,0.773566,0.402912
1,Prom Night,0.022222,0.773566,0.397894
2,Ludwig,0.033333,0.472718,0.253026
3,Yamakasi - Les samouraïs des temps modernes,0.125000,0.377145,0.251072
4,El asaltante,0.040000,0.446949,0.243475
5,Midnight in the Garden of Good and Evil,0.111111,0.373841,0.242476
6,Sneakers,0.068966,0.415830,0.242398
7,Topaz,0.103448,0.377145,0.240296
8,Godzilla,0.052632,0.413141,0.232886
9,Bushwhacked,0.064516,0.396081,0.230299
